# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# current working directory
print(os.getcwd())

# current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 #  data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
             
#print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print("Connected")
except Exception as e:
    print(e)

Connected


#### Create Keyspace

In [6]:
try:
    session.execute("""create keyspace if not exists udacity_cassandra_project 
    with replication =
     { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
    print("Keyspace created")
except Exception as e:
    print(e)
    

Keyspace created


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity_cassandra_project')
    print("Keyspace set")
except Exception as e:
    print(e)

Keyspace set


### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create a table with a primary key composite from 2 features: session_id, item_in_session

In [8]:
query_create_songs = "create table if not exists songs(artist text, song_title text, length float, level text, session_id int, item_in_session int, primary key(session_id, item_in_session))"
try:
    session.execute(query_create_songs)
    print("Songs table created")
except Exception as e:
    print(e)

Songs table created


### Insert data into songs table from csv file

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into songs (artist, song_title, length, level, session_id, item_in_session)"
        query = query + "values(%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), line[6],int(line[8]), int(line[3])))

### Verify that the data have been inserted into songs table

In [10]:
query_check = "select * from songs limit 10"
try:
    rows_select = session.execute(query_check)
except Exception as e:
    print(e)
for row in rows_select:
    print(row)

Row(session_id=23, item_in_session=0, artist='Regina Spektor', length=191.08526611328125, level='paid', song_title='The Calculation (Album Version)')
Row(session_id=23, item_in_session=1, artist='Octopus Project', length=250.95791625976562, level='paid', song_title='All Of The Champs That Ever Lived')
Row(session_id=23, item_in_session=2, artist='Tegan And Sara', length=180.06158447265625, level='paid', song_title='So Jealous')
Row(session_id=23, item_in_session=3, artist='Dragonette', length=153.39056396484375, level='paid', song_title='Okay Dolores')
Row(session_id=23, item_in_session=4, artist='Lil Wayne / Eminem', length=229.58975219726562, level='paid', song_title='Drop The World')
Row(session_id=23, item_in_session=5, artist="Soulja Boy Tell'em", length=201.1162872314453, level='paid', song_title='Let Me Get Em')
Row(session_id=23, item_in_session=6, artist='Bodo Wartke', length=645.2763061523438, level='paid', song_title='Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃ\x83Â¶sisc

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
query1 = "select artist, song_title, length from songs where session_id = 338 and item_in_session = 4"
try:
    rows1 = session.execute(query1)
    for row in rows1:
        print(row.artist, row.song_title, row.length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Create a table with a composite primary key and a cluster to order the result of select by the column item_in_session

In [12]:
query_create_song_session = "create table if not exists song_session(artist text, song_title text, first_name_user text, last_name_user text, user_id int, session_id int, item_in_session int, primary key((user_id, session_id), item_in_session)) with clustering order by (item_in_session asc)"
try:
    session.execute(query_create_song_session)
    print("Song_Session table created")
except Exception as e:
    print(e)

Song_Session table created


### Insert data into songs_session table from csv file

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_session(artist, song_title, first_name_user, last_name_user, user_id, session_id, item_in_session)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

#### Verify that the data have been inserted into song_session table

In [14]:
query_select_song_session = "select * from song_session limit 10"
try:
    rows_select = session.execute(query_select_song_session)
    for row in rows_select:
        print(row)
except Exception as e:
    print(e)

Row(user_id=58, session_id=768, item_in_session=0, artist='System of a Down', first_name_user='Emily', last_name_user='Benson', song_title='Sad Statue')
Row(user_id=58, session_id=768, item_in_session=1, artist='Ghostland Observatory', first_name_user='Emily', last_name_user='Benson', song_title='Stranger Lover')
Row(user_id=58, session_id=768, item_in_session=2, artist='Evergreen Terrace', first_name_user='Emily', last_name_user='Benson', song_title='Zero')
Row(user_id=85, session_id=776, item_in_session=2, artist='Deftones', first_name_user='Kinsley', last_name_user='Young', song_title='Head Up (LP Version)')
Row(user_id=85, session_id=776, item_in_session=3, artist='The Notorious B.I.G.', first_name_user='Kinsley', last_name_user='Young', song_title='Playa Hater (Amended Version)')
Row(user_id=85, session_id=776, item_in_session=4, artist='Orchestral Manoeuvres In The Dark', first_name_user='Kinsley', last_name_user='Young', song_title='Stay (The Black Rose And The Universal Wheel)'

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 

In [15]:
query2 ="select artist, song_title, item_in_session, first_name_user, last_name_user from song_session where user_id = 10 and session_id = 182 order by item_in_session" 
try:
    rows2 = session.execute(query2)
    for row in rows2:
        print(row.artist, row.song_title,row.item_in_session,row.first_name_user, row.last_name_user)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


### For the third query, we need to create a composite primary key being song_title column and using a cluster user_id column

In [16]:
query_create_users = "create table if not exists users(first_name text, last_name text, song_title text,user_id int, primary key((song_title),user_id))"
try:
    session.execute(query_create_users)
    print("Users table created")
except Exception as e:
    print(e)

Users table created


### Insert data into users table from csv file

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into users(first_name, last_name, song_title, user_id)"
        query = query + "values(%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9],int(line[10])))

### Verify that the data have been inserted into users table

In [18]:
query_select_users = "select * from users limit 10"
try:
    rows_select = session.execute(query_select_users)
    for row in rows_select:
        print(row.first_name, row.last_name)
except Exception as e:
    print(e)

Chloe Cuevas
Chloe Cuevas
Aleena Kirby
Chloe Cuevas
Lily Koch
Jacob Klein
Layla Griffin
Tegan Levine
Mohammad Rodriguez
Kate Harrell


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [19]:
query3 = "select first_name, last_name from users where song_title = 'All Hands Against His Own'"
try:
    rows3 = session.execute(query3)
    for row in rows3:
        print(row.first_name, row.last_name)
except Exception as e:
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

### Drop the first table

In [20]:
query_delete_1="drop table if exists songs"
try:
    session.execute(query_delete_1)
    print("Songs table deleted succes successfully")
except Exception as e:
    print(e)

Songs table deleted succes successfully


### Drop the second table

In [21]:
query_delete_2="drop table if exists song_session"
try:
    session.execute(query_delete_2)
    print("Song_session table deleted successfully")
except Exception as e:
    print(e)

Song_session table deleted successfully


### Drop the third table

In [22]:
query_delete_3="drop table if exists users"
try:
    session.execute(query_delete_2)
    print("Users table deleted successfully")
except Exception as e:
    print(e)

Users table deleted successfully


### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()